# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [12]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [38]:
# set up environment
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

openai = OpenAI()

API key looks good so far


In [15]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [16]:
dr = Website("https://www.drbruceforciea.com")
print(dr.get_contents())
print(dr.links)

Webpage Title:
Home
Webpage Contents:
Home
Custom AI Chatbots
Educator's Guide to AI Resources
Contact
Media
Books
Anatomy and Physiology
Interactive Learning Activities for A&P
3D Models for Anatomy & Physiology
Infographics for Learning A&P
Presentations
Dr. Bruce Forciea
Elearning Designer /Author/Educator/SME
Books
Elearning, Educator, Author, GenAI
Media
Welcome to my page. Here you will find a variety of resources along with my books and media samples. ​
Bottom line is I teach and develop courses in Anatomy and Physiology and Pathophysiology. I am passionate about making online courses as easy to use as possible by making use of principles of usability and use of mutimodal microlearning media. I am also a big AI enthusiast and evangelist.
I'm also a science fiction author and have written 2 novels and a series of short stories.
Discover the transformative power of AI in education with "An Educator’s Guide to AI.” Written by a college instructor with over 20 years experience in te

In [17]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to learn anatomy and physiology, \
such as links to an Anatomy or Physiology page, Learing Page, Book Page.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "anatomy and physiology page", "url": "https://full.url/goes/here/anatomy-and-physiology"},
        {"type": "learning page": "url": "https://another.full.url/learning"}
    ]
}
"""

In [18]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links to learn anatomy and physiology, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [19]:
print(get_links_user_prompt(dr))

Here is the list of links on the website of https://www.drbruceforciea.com - please decide which of these are relevant web links to learn anatomy and physiology, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
home.html
custom-ai-chatbots.html
educator-s-guide-to-ai-resources.html
contact.html
media.html
books.html
anatomy-and-physiology.html
interactive-learning-activities-for-a-p.html
3d-models-for-anatomy---physiology.html
infographics-for-learning-a-p.html
presentations.html
books.html
https://www.amazon.com/Educators-Guide-Transforming-Artificial-Intelligence-ebook/dp/B0D54T95YJ/ref=sr_1_3?crid=39616Q2815N4Z&dib=eyJ2IjoiMSJ9.xxyfMRpsHyy-k63PCb7bUMg7RfXPZNZjDLuEfpat4cWBwJErNmtHkbUjkbn55Kqu-FzR3fs2lZjPYbNdkA0iHFxK6D4obsb1jzMFhXRWuxssxrG8EWDrh60x3DwD0poM-Wq5eHpgIQfbWdaplya5cAtmceMJm-AmEjKy_tVHkzuodeBicYeo87M221aGTc4sX4LPeLRzuc2WCBQRxSWhhEzeOGJAUPJJdXHgC377YXji0DIpT_-HNhjDGo4D22EiyYOhWsn2ptECC

In [34]:
# Give a medicine related website link.

nationalcancerinstitute = Website("https://training.seer.cancer.gov/modules_reg_surv.html")
nationalcancerinstitute.links

['#skip',
 '/index.html',
 '/index.html',
 '/citation.html',
 '/help/',
 '/',
 '/modules_reg_surv.html',
 '/registration/',
 '/operations/',
 '/disease/',
 '/terminology/',
 '/anatomy/',
 '/casefinding/',
 '/icd10cm/',
 '/abstracting/',
 '/diagnostic/',
 '/coding-primary/',
 '/staging/',
 '/ss2k/',
 '/treatment/',
 '/followup/',
 '/eod/',
 '/modules_site_spec.html',
 '/modules_info.html',
 '/modules_arch.html',
 '/modules_update.html',
 '/modules_ack.html',
 'https://twitter.com/NCICancerStats',
 'https://surveillance.cancer.gov/blog',
 '/contact.html',
 'https://livehelp.cancer.gov/',
 '/index.html',
 '/citation.html',
 '/help/',
 'https://seer.cancer.gov/',
 'https://www.cancer.gov/policies/accessibility',
 'https://www.cancer.gov/policies/disclaimer',
 'https://www.cancer.gov/policies/foia',
 'https://www.hhs.gov/vulnerability-disclosure-policy/index.html',
 'https://www.cancer.gov/policies/privacy-security',
 'https://www.cancer.gov/policies/copyright-reuse',
 'https://www.cancer.g

In [39]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [86]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [43]:
get_all_details("https://training.seer.cancer.gov/anatomy/")

Found links: {'links': [{'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/anatomy/'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/anatomy/body/'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/anatomy/body/functions.html'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/anatomy/body/terminology.html'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/anatomy/body/review.html'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/anatomy/cells_tissues_membranes/'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/anatomy/cells_tissues_membranes/cells/'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/anatomy/cells_tissues_membranes/cells/structure.html'}, {'type': 

'Landing page:\nWebpage Title:\nAnatomy & Physiology | SEER Training\nWebpage Contents:\nSkip to Main Content\nAn official website of the United States government\nSEER Training Modules\nSearch SEER Training:\nSearch\nMain Menu\nHome\nCitation\nHelp\nSearch\nThe SEER Training Website is a training resource for oncology data specialists (ODS) and cancer registration trainees. It is not intended to provide medical advice or to guide clinical care, diagnosis, or treatment. NCI urges users to consult with a qualified physician for diagnosis and for answers to their personal medical questions.\nHome\n»\nCancer Registration & Surveillance Modules\n»\nAnatomy & Physiology\nSection Menu\nCancer Registration & Surveillance Modules\nAnatomy & Physiology\nIntro to the Human Body\nBody Functions & Life Process\nAnatomical Terminology\nReview\nCells, Tissues, & Membranes\nCell Structure & Function\nCell Structure\nCell Function\nBody Tissues\nEpithelial Tissue\nConnective Tissue\nMuscle Tissue\nNer

# A Learning Notebook Agent

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a learning notebook from a research institue web page to learn anatomy and physiology for prospective medical school students.

We will be provided a research institue page and their primary website.

See the end of this notebook for examples of real-world business applications.

In [159]:
system_prompt = "You are an eLearning designer, author, educator, and subject matter expert specializing in Anatomy, Physiology, and Pathophysiology.\
With over 20 years of experience, you are dedicated to enhancing online education by applying usability principles and multimodal microlearning media.\
you analyzes the contents of several relevant pages from a research institue website \
and creates a learning notebook of anatamy and physiology for prospective medical school students. \
Add url links to the related terminology as much as possible.\
Include details of explanation of anatamy terminology if you have the information. \
Respond in markdown."

In [160]:
def get_learning_notebook_user_prompt(research_institue, url):
    user_prompt = f"You are looking at a research institue called: {research_institue}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short learning notebook of the research institue in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:1_0000] # Truncate if more than 10,000 characters
    return user_prompt

In [161]:
get_learning_notebook_user_prompt("National Cancer Institue", "https://training.seer.cancer.gov/anatomy/")

Found links: {'links': [{'type': 'anatomy page', 'url': 'https://training.seer.cancer.gov/anatomy/'}, {'type': 'anatomy body page', 'url': 'https://training.seer.cancer.gov/anatomy/body/'}, {'type': 'anatomy body functions page', 'url': 'https://training.seer.cancer.gov/anatomy/body/functions.html'}, {'type': 'anatomy body terminology page', 'url': 'https://training.seer.cancer.gov/anatomy/body/terminology.html'}, {'type': 'anatomy body review page', 'url': 'https://training.seer.cancer.gov/anatomy/body/review.html'}, {'type': 'anatomy cells, tissues, membranes page', 'url': 'https://training.seer.cancer.gov/anatomy/cells_tissues_membranes/'}, {'type': 'anatomy cells page', 'url': 'https://training.seer.cancer.gov/anatomy/cells_tissues_membranes/cells/'}, {'type': 'anatomy cells structure page', 'url': 'https://training.seer.cancer.gov/anatomy/cells_tissues_membranes/cells/structure.html'}, {'type': 'anatomy cells function page', 'url': 'https://training.seer.cancer.gov/anatomy/cells_t

'You are looking at a research institue called: National Cancer Institue\nHere are the contents of its landing page and other relevant pages; use this information to build a short learning notebook of the research institue in markdown.\nLanding page:\nWebpage Title:\nAnatomy & Physiology | SEER Training\nWebpage Contents:\nSkip to Main Content\nAn official website of the United States government\nSEER Training Modules\nSearch SEER Training:\nSearch\nMain Menu\nHome\nCitation\nHelp\nSearch\nThe SEER Training Website is a training resource for oncology data specialists (ODS) and cancer registration trainees. It is not intended to provide medical advice or to guide clinical care, diagnosis, or treatment. NCI urges users to consult with a qualified physician for diagnosis and for answers to their personal medical questions.\nHome\n»\nCancer Registration & Surveillance Modules\n»\nAnatomy & Physiology\nSection Menu\nCancer Registration & Surveillance Modules\nAnatomy & Physiology\nIntro to 

In [162]:
def create_learning_notebook(research_institue_name, url, display_content):
    response = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_learning_notebook_user_prompt(research_institue_name, url)}
          ],
    )
    result = response.choices[0].message.content
    if display_content:
        display(Markdown(result))
    else :
        return result


In [163]:
create_learning_notebook("NationalCancerInstitue", "https://training.seer.cancer.gov/anatomy/", True)

Found links: {'links': [{'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/body/'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/body/functions.html'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/body/terminology.html'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/body/review.html'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/cells_tissues_membranes/'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/cells_tissues_membranes/cells/'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/cells_tissues_membranes/cells/structure.html'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.can

# Learning Notebook: Anatomy & Physiology at the National Cancer Institute

## Overview
The **SEER Training** website by the National Cancer Institute (NCI) offers comprehensive modules on Anatomy & Physiology tailored for oncology data specialists and cancer registration trainees. This training resource covers various aspects of human anatomy, physiological processes, and their relevance to cancer studies. This notebook aims to summarize the key topics covered in the training modules, along with terminology that prospective medical school students should be familiar with.

## Core Topics

### 1. Introduction to the Human Body
- **Basic Human Body Functions**: Functions vital for sustaining life.
- **Major Human Body Systems**: Overview of systems such as the skeletal, muscular, nervous, and circulatory systems.
- **Anatomical Locations**: Understanding the position of organs and body parts.

**Key Resources**: [Intro to the Human Body](https://seer.cancer.gov/seertraining/modules/anatomy/intro.html)

### 2. Cells, Tissues, & Membranes
- **Cell Structure & Function**: Explains the basic unit of life and its functions.
- **Body Tissues**:
  - **Epithelial Tissue**: Covers body surfaces; forms glands.
  - **Connective Tissue**: Supports, binds, and protects tissues.
  - **Muscle Tissue**: Responsible for movement.
  - **Nervous Tissue**: Processes information and controls responses.

**Key Resources**: [Cells, Tissues, & Membranes](https://seer.cancer.gov/seertraining/modules/anatomy/cells.html)

### 3. Skeletal System
- **Bone Structure and Function**: Understanding the framework of the body.
- **Axial and Appendicular Skeleton**:
  - **Axial Skeleton**: Comprises 80 bones, including the skull and spine.
  - **Appendicular Skeleton**: Comprises 126 bones, including limbs and girdles.

**Key Resources**: [Skeletal System Overview](https://seer.cancer.gov/seertraining/modules/anatomy/skeletal.html)

### 4. Muscular System
- **Muscle Types**: Includes skeletal, cardiac, and smooth muscle.
- **Muscle Groups**: Locations and functions of muscular groups in the body.

**Key Resources**: [Muscular System Overview](https://seer.cancer.gov/seertraining/modules/anatomy/muscular.html)

### 5. Nervous System
- **Central and Peripheral Nervous System**: Breakdown of nerve functions and structures.

**Key Resources**: [Nervous System Overview](https://seer.cancer.gov/seertraining/modules/anatomy/nervous.html)

### 6. Endocrine System
- **Hormones**: Characteristics and functions of hormones produced by glands such as the pituitary, thyroid, and adrenal glands.

**Key Resources**: [Endocrine System Overview](https://seer.cancer.gov/seertraining/modules/anatomy/endocrine.html)

### 7. Cardiovascular System
- **Heart and Circulation**: Anatomy and physiology of the heart, vessels, and blood circulation.

**Key Resources**: [Cardiovascular System Overview](https://seer.cancer.gov/seertraining/modules/anatomy/cardiovascular.html)

### 8. Lymphatic System
- **Components**: Includes lymph nodes, spleen, and thymus.

**Key Resources**: [Lymphatic System Overview](https://seer.cancer.gov/seertraining/modules/anatomy/lymphatic.html)

### 9. Respiratory System
- **Ventilation Mechanics**: Understanding how air is exchanged in the lungs.

**Key Resources**: [Respiratory System Overview](https://seer.cancer.gov/seertraining/modules/anatomy/respiratory.html)

### 10. Digestive System
- **Digestive Organs**: From the mouth to the intestines, including accessory organs.

**Key Resources**: [Digestive System Overview](https://seer.cancer.gov/seertraining/modules/anatomy/digestive.html)

### 11. Urinary System
- **Components**: Functions of the kidneys, ureters, bladder, and urethra.

**Key Resources**: [Urinary System Overview](https://seer.cancer.gov/seertraining/modules/anatomy/urinary.html)

### 12. Reproductive System
- **Male and Female Systems**: Anatomy and functions related to reproduction.

**Key Resources**: [Reproductive System Overview](https://seer.cancer.gov/seertraining/modules/anatomy/reproductive.html)

## Conclusion
This learning notebook highlights essential aspects of Anatomy and Physiology as presented on the SEER Training website by the National Cancer Institute. For further exploration, prospective medical students are encouraged to engage with the training modules linked above, which provide in-depth knowledge and understanding necessary for a career in medicine.

**Visit the full SEER Training resource here**: [SEER Training Modules](https://seer.cancer.gov/seertraining/modules/)

---
This notebook is designed to serve as a foundational guide for those interested in understanding human anatomy and physiology, particularly in the context of cancer research and diagnosis.

In [191]:
# Provide learning notebook content to be translate later
content_page_to_be_translate=create_learning_notebook("NationalCancerInstitue", "https://training.seer.cancer.gov/anatomy/body/terminology.html", False)

Found links: {'links': [{'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/body/functions.html'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/body/review.html'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/cells_tissues_membranes/'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/cells_tissues_membranes/cells/'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/cells_tissues_membranes/tissues/'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/skeletal/'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/muscular/'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anatomy/nervous/'}, {'type': 'anatomy and physiology page', 'url': 'https://training.seer.cancer.gov/anat

In [192]:
system_translation_prompt = "You are an English to Chinese translation expert who has all the skills the same as google translate.\
you also have the domain knowledge of Anatomy and Physiology, \
you translates the contents of the research institue English page, \
use the information to build a translated version in both English and Chinese, \
make the Chinese version under the English version, \
keep the original url links in the English version."

In [193]:
def get_learning_notebook_translate_user_prompt(english_page, content):
    user_prompt = f"You are translating this English page: {english_page}\n"
    user_prompt += f"Here are the contents of the English page; \
    use this information to build a translated version in Both English and Chinese."
    user_prompt += content
    user_prompt = user_prompt[:1_0000] # Truncate if more than 10,000 characters
    return user_prompt

In [194]:
def create_translate_page(name, content):
    response = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_translation_prompt},
            {"role": "user", "content": get_learning_notebook_translate_user_prompt(name, content)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [195]:
create_translate_page("Learning Notebook: National Cancer Institute - Anatomy & Physiology", content_page_to_be_translate)

# Learning Notebook: Anatomy and Physiology from the National Cancer Institute

Welcome to your learning notebook for understanding the foundational concepts of anatomy and physiology, especially in the context of oncology and cancer registration. This information is compiled from the National Cancer Institute's SEER Training resources.

# 学习笔记本：来自国家癌症研究所的解剖学和生理学

欢迎来到您的学习笔记本，以理解解剖学和生理学的基础概念，尤其是在肿瘤学和癌症登记的背景下。该信息来自国家癌症研究所的SEER培训资源。

## 1. Introduction to Anatomical Terminology

Anatomical terminology is essential for accurately describing body structures and their locations. Understanding these terms will aid comprehension of how different body systems work together.

## 1. 解剖学术语介绍

解剖学术语对于准确描述身体结构及其位置至关重要。理解这些术语将有助于理解不同的身体系统如何共同工作。

### Directional Terms
- **Superior (Cranial)**: Toward the head (e.g., the hand is part of the superior extremity).
- **Inferior (Caudal)**: Away from the head (e.g., the foot is part of the inferior extremity).
- **Anterior (Ventral)**: Front (e.g., the kneecap is on the anterior side of the leg).
- **Posterior (Dorsal)**: Back (e.g., the shoulder blades are on the posterior side of the body).
- **Medial**: Toward the midline (e.g., the middle toe is on the medial side of the foot).
- **Lateral**: Away from the midline (e.g., the little toe is on the lateral side of the foot).
- **Proximal**: Nearest to the trunk (e.g., the proximal end of the femur joins with the pelvic bone).
- **Distal**: Farthest from the trunk (e.g., the hand is at the distal end of the forearm).

### 方向术语
- **上方（头侧）**: 指向头部（例如，手是上肢的一部分）。
- **下方（尾侧）**: 远离头部（例如，脚是下肢的一部分）。
- **前方（腹侧）**: 前面（例如，膝盖骨在腿的前侧）。
- **后方（背侧）**: 后面（例如，肩胛骨在身体的后侧）。
- **内侧**: 朝向中线（例如，中间的脚趾在脚的内侧）。
- **外侧**: 远离中线（例如，小脚趾在脚的外侧）。
- **近端**: 最接近躯干（例如，股骨的近端与骨盆骨连接）。
- **远端**: 离躯干最远（例如，手位于前臂的远端）。

### Planes of the Body
- **Coronal Plane (Frontal Plane)**: Divides the body into anterior and posterior portions.
- **Sagittal Plane (Lateral Plane)**: Divides the body into right and left sides.
- **Axial Plane (Transverse Plane)**: Divides the body into upper and lower parts.

### 身体的平面
- **冠状面（前面）**: 将身体分为前部和后部。
- **矢状面（侧面）**: 将身体分为左右两侧。
- **轴面（横截面）**: 将身体分为上部和下部。

### Body Cavities
- **Ventral Cavity**: Larger cavity subdivided into thoracic and abdominopelvic cavities.
  - **Thoracic Cavity**: Contains the heart, lungs, trachea, and esophagus.
  - **Abdominal Cavity**: Houses most of the gastrointestinal tract and kidneys.
  - **Pelvic Cavity**: Contains the urogenital system and rectum.
- **Dorsal Cavity**: Smaller cavity divided into cranial and vertebral portions.
  - **Cranial Cavity**: Houses the brain.
  - **Vertebral Canal**: Contains the spinal cord.

### 身体腔
- **腹腔**: 较大的腔室，分为胸腔和腹盆腔。
  - **胸腔**: 包含心脏、肺、气管和食道。
  - **腹腔**: 存放大部分消化道和肾脏。
  - **盆腔**: 包含泌尿生殖系统和直肠。
- **背腔**: 较小的腔室，分为颅腔和脊柱部分。
  - **颅腔**: 包含大脑。
  - **脊髓管**: 包含脊髓。

For further reading on anatomical terminology, visit [SEER Training: Anatomical Terminology](https://seer.cancer.gov/).

有关解剖学术语的更多阅读信息，请访问 [SEER培训：解剖学术语](https://seer.cancer.gov/)。

## 2. Body Functions & Life Process

The body's functions are driven by physiological and psychological processes at the cellular level, forming the basis of how each system operates.

## 2. 身体功能与生命过程

身体的功能由细胞水平的生理和心理过程驱动，构成了每个系统运作的基础。

### Body Functions
- **Cellular Function**: Each function of the body is ultimately a function of its cells. These include both physiological actions (such as digestion and movement) and psychological functions (such as mood and cognitive processing).
- **Survival Mechanisms**: The body systems work synergistically to maintain homeostasis and enable survival through various physical and metabolic processes.

### 身体功能
- **细胞功能**: 身体的每个功能最终都是其细胞的功能。这些包括生理行为（如消化和运动）和心理功能（如情绪和认知处理）。
- **生存机制**: 身体系统协同工作以维持体内平衡，并通过各种生理和代谢过程实现生存。

For more details about body functions and life processes, explore [SEER Training: Body Functions & Life Processes](https://seer.cancer.gov/).

有关身体功能和生命过程的更多细节，请查阅 [SEER培训：身体功能与生命过程](https://seer.cancer.gov/)。

## 3. Key Systems to Explore

- **Skeletal System**: Understand bone structure, types, and functions.
- **Muscular System**: Learn about muscle types and their roles in movement.
- **Nervous System**: Delve into the organization and functions of the nervous system.
- **Endocrine System**: Explore hormone characteristics and regulatory functions.
- **Cardiovascular System**: Understand heart structure, function, and blood circulation.
- **Lymphatic System**: Learn about the components and roles in immunity.
- **Respiratory System**: Review mechanics of breathing and respiratory anatomy.
- **Digestive System**: Study the gastrointestinal tract and digestive processes.
- **Urinary System**: Understand kidney functions and waste elimination.
- **Reproductive System**: Learn about male and female reproductive anatomy and functions.

## 3. 关键系统探索

- **骨骼系统**: 理解骨骼结构、类型和功能。
- **肌肉系统**: 了解肌肉类型及其在运动中的作用。
- **神经系统**: 深入了解神经系统的组织和功能。
- **内分泌系统**: 探索激素特征和调节功能。
- **心血管系统**: 理解心脏结构、功能及血液循环。
- **淋巴系统**: 了解组成部分及其在免疫中的作用。
- **呼吸系统**: 回顾呼吸的机制和呼吸解剖。
- **消化系统**: 研究消化道和消化过程。
- **泌尿系统**: 理解肾脏功能和废物排泄。
- **生殖系统**: 了解男性和女性生殖解剖和功能。

## 4. Resources and Further Learning

The SEER Training website offers a wealth of modules covering anatomy, physiology, and cancer registration. Users are encouraged to consult the information provided, but always seek guidance from healthcare professionals for personal medical inquiries.

## 4. 资源与进一步学习

SEER培训网站提供了丰富的模块，涵盖解剖学、生理学和癌症登记。建议用户咨询所提供的信息，但始终应向医疗专业人士寻求个人医疗咨询的指导。

- Access the main SEER Training Website: [SEER Training](https://seer.cancer.gov/)
- Contact NCI LiveHelp for further assistance: [NCI LiveHelp](https://livehelp.cancer.gov/)

- 访问主要的SEER培训网站: [SEER培训](https://seer.cancer.gov/)
- 联系NCI LiveHelp获取更多帮助: [NCI LiveHelp](https://livehelp.cancer.gov/)

This notebook serves as a foundational guide for prospective medical school students as they prepare for more advanced studies in anatomy and physiology.

本笔记本旨在为有意考取医学院的学生提供基础指南，以便他们为更高级别的解剖学和生理学研究做准备。

## Add Gradio UI

In [197]:
import gradio as gr

In [203]:
def create_learning_notebook_without_format(research_institue_name, url):
    response = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_learning_notebook_user_prompt(research_institue_name, url)}
          ],
    )
    result = response.choices[0].message.content
    return result

In [205]:
def summary_and_translate_anatomy_online_resources_stream_gpt(name, url):
    messages=[
        {"role": "system", "content": system_translation_prompt},
        {"role": "user", "content": get_learning_notebook_translate_user_prompt(name, create_learning_notebook_without_format(name, url))}
    ]
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [209]:
view = gr.Interface(
    fn=summary_and_translate_anatomy_online_resources_stream_gpt,
    inputs=[
        gr.Textbox(label="Anatomy Online Resource:"),
        gr.Textbox(label="Anatomy Online resource page URL including http:// or https://")],
    outputs=[gr.Markdown(label="Anatomy Summary & Translation:")],
    flagging_mode="never"
)
view.launch(share=True)

* Running on local URL:  http://127.0.0.1:7876
* Running on public URL: https://efca7dbc932ecd5a15.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Found links: {'links': [{'type': 'anatomy page', 'url': 'https://training.seer.cancer.gov/anatomy/'}, {'type': 'body anatomy page', 'url': 'https://training.seer.cancer.gov/anatomy/body/'}, {'type': 'functions page', 'url': 'https://training.seer.cancer.gov/anatomy/body/functions.html'}, {'type': 'terminology page', 'url': 'https://training.seer.cancer.gov/anatomy/body/terminology.html'}, {'type': 'review page', 'url': 'https://training.seer.cancer.gov/anatomy/body/review.html'}, {'type': 'cells, tissues, membranes page', 'url': 'https://training.seer.cancer.gov/anatomy/cells_tissues_membranes/'}, {'type': 'skeletal system page', 'url': 'https://training.seer.cancer.gov/anatomy/skeletal/'}, {'type': 'muscular system page', 'url': 'https://training.seer.cancer.gov/anatomy/muscular/'}, {'type': 'nervous system page', 'url': 'https://training.seer.cancer.gov/anatomy/nervous/'}, {'type': 'endocrine system page', 'url': 'https://training.seer.cancer.gov/anatomy/endocrine/'}, {'type': 'cardi

In [206]:
summary_and_translate_anatomy_online_resources_stream_gpt("Learning Notebook: National Cancer Institute - Anatomy & Physiology", "https://training.seer.cancer.gov/anatomy/body/terminology.html")

<generator object summary_and_translate_anatomy_online_resources_stream_gpt at 0x12f55e200>

# A Terminolgy Agent

## Now we will take our project from Day 1 to the next next level!

### BUSINESS CHALLENGE:

Create a terminology list table for a research institue page to learn anatomy and physiology terms.

We will be provided a research institue page and their primary website.

We also provide a chinese version for the terminology

See the end of this notebook for examples of real-world business applications.

In [112]:
def get_all_details_from_page(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    # print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [120]:
system_terminology_prompt = "You are an eLearning designer, author, educator, and subject matter expert specializing in Anatomy, Physiology, and Pathophysiology.\
With over 20 years of experience, you are dedicated to enhancing online education by applying usability principles and multimodal microlearning media.\
you analyzes the contents of several relevant pages from a research institue website \
and generate a full list of anatamy terminology for prospective medical school students.\
Add Anatomical Term and Regional Term if you have the information.\
Include details of explanation of anatamy terminology if you have the information.\
Respond in a table format."

In [121]:
def get_terminology_list_user_prompt(research_institue, url):
    user_prompt = f"You are looking at a research institue called: {research_institue}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a full list of anatomy and physiology terminology of the research institue page in a table format.\n"
    user_prompt += get_all_details_from_page(url)
    user_prompt = user_prompt[:1_0000] # Truncate if more than 5,000 characters
    return user_prompt

In [122]:
get_terminology_list_user_prompt("National Cancer Institue", "https://training.seer.cancer.gov/anatomy/")

'You are looking at a research institue called: National Cancer Institue\nHere are the contents of its landing page and other relevant pages; use this information to build a full list of anatomy and physiology terminology of the research institue page in a table format.\nLanding page:\nWebpage Title:\nAnatomy & Physiology | SEER Training\nWebpage Contents:\nSkip to Main Content\nAn official website of the United States government\nSEER Training Modules\nSearch SEER Training:\nSearch\nMain Menu\nHome\nCitation\nHelp\nSearch\nThe SEER Training Website is a training resource for oncology data specialists (ODS) and cancer registration trainees. It is not intended to provide medical advice or to guide clinical care, diagnosis, or treatment. NCI urges users to consult with a qualified physician for diagnosis and for answers to their personal medical questions.\nHome\n»\nCancer Registration & Surveillance Modules\n»\nAnatomy & Physiology\nSection Menu\nCancer Registration & Surveillance Modul

In [124]:
def create_terminology_list(research_institue_name, url, display_result):
    response = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_terminology_prompt},
            {"role": "user", "content": get_terminology_list_user_prompt(research_institue_name, url)}
          ],
    )
    result = response.choices[0].message.content
    if display_result:
        display(Markdown(result))
    else :
        return result

In [125]:
create_terminology_list("National Cancer Institue", "https://training.seer.cancer.gov/anatomy/body/terminology.html", True)

Here's a table of anatomy and physiology terminology based on the contents of the National Cancer Institute's SEER Training page. This table includes the key anatomical terms, regional terms, and their explanations.

| **Term Category**       | **Term**               | **Explanation**                                                                                        |
|-------------------------|-----------------------|--------------------------------------------------------------------------------------------------------|
| **Directional Terms**   | Superior               | Toward the head end of the body; upper (e.g., the hand is superior to the foot).                      |
|                         | Inferior              | Away from the head; lower (e.g., the foot is inferior to the knee).                                   |
|                         | Anterior (Ventral)    | Front (e.g., the kneecap is located on the anterior side of the leg).                                |
|                         | Posterior (Dorsal)    | Back (e.g., the shoulder blades are located on the posterior side of the body).                      |
|                         | Medial                | Toward the midline of the body (e.g., the middle toe is medial to the little toe).                   |
|                         | Lateral               | Away from the midline of the body (e.g., the little toe is lateral to the middle toe).               |
|                         | Proximal              | Toward or nearest the trunk or point of origin of a part (e.g., the proximal end of the femur).      |
|                         | Distal                | Away from or farthest from the trunk or point of origin of a part (e.g., the hand is distal to the elbow). |
| **Planes of the Body**  | Coronal (Frontal)     | A vertical plane running from side to side; divides the body into anterior and posterior portions.     |
|                         | Sagittal (Lateral)     | A vertical plane running from front to back; divides the body into right and left parts.              |
|                         | Axial (Transverse)   | A horizontal plane; divides the body into upper and lower parts.                                      |
|                         | Median                 | A sagittal plane through the midline of the body; divides the body into right and left halves.      |
| **Body Cavities**      | Ventral               | The larger cavity; subdivided into thoracic and abdominopelvic cavities by the diaphragm.             |
|                         | Thoracic Cavity       | Contains the heart, lungs, trachea, esophagus, and large blood vessels.                               |
|                         | Abdominal Cavity      | Contains most of the gastrointestinal tract and organs such as the kidneys and adrenal glands.       |
|                         | Pelvic Cavity         | Contains the urogenital system and the rectum.                                                        |
|                         | Dorsal Cavity         | Contains organs lying more posterior; comprised of cranial and vertebral cavities.                    |
|                         | Cranial Cavity        | Houses the brain.                                                                                       |
|                         | Vertebral Canal       | Contains the spinal cord.                                                                              |
| **System Terms**    | Skeletal System       | Composed of bones; provides structure and support to the body.                                         |
|                         | Muscular System       | Consists of skeletal muscles; allows for movement and stability.                                        |
|                         | Nervous System        | Comprises the brain, spinal cord, and nerves; controls and communicates through signaling.             |
|                         | Endocrine System      | Involves glands that secrete hormones to regulate bodily functions.                                    |
|                         | Cardiovascular System  | Consists of the heart and blood vessels; responsible for circulation of blood.                         |
|                         | Lymphatic System      | Components including lymph nodes and vessels; part of the immune system.                               |
|                         | Respiratory System    | Involved in gas exchange; includes nose, lungs, and trachea.                                          |
|                         | Digestive System      | Responsible for breakdown and absorption of nutrients; includes stomach and intestines.                |
|                         | Urinary System        | Comprises kidneys and bladder; removes waste and maintains water balance.                                |
|                         | Reproductive System    | Involves organs such as ovaries, testes; responsible for reproduction.                                |

This terminalogy table provides a comprehensive overview for prospective medical students, aiding in their understanding of basic anatomical and physiological terms relevant to studying human body systems.

In [145]:
terminology_table_to_be_translate = create_terminology_list("National Cancer Institue", "https://training.seer.cancer.gov/anatomy/body/terminology.html", False)

In [148]:
system_terminology_translation_prompt = "You are an Englisht to Chinese translation expert who has all the skills the same as google translate.\
you also have the domain knowledge of Anatomy, \
first you add a new column to add Phonetic symbols and embed audio links for the Term column, \
then you translate the Term column into Chinese as another new Column, \
keep other column untouched and in the original format."

In [149]:
def get_terminology_list_translate_user_prompt(english_page, terminology_table_content):
    user_prompt = f"You are translating this Anatomy terminolgy English page: {english_page}\n"
    user_prompt += f"Here are the contents of this Anatomy terminolgy table; use this information to build \
    a translated version of Term column, \
    add a new column to add Phonetic symbols and embed audio links for the Term column, \
    translate the Term column into Chinese as another new Column, \
    keep other column untouched and in the original format,\
    keep the full output in a table format.\n"
    user_prompt += terminology_table_content
    user_prompt = user_prompt[:1_0000] # Truncate if more than 5,000 characters
    return user_prompt

In [150]:
def create_translate_terminology_table(name, content):
    response = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_terminology_translation_prompt},
            {"role": "user", "content": get_terminology_list_translate_user_prompt(name, content)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [151]:
create_translate_terminology_table("Anatomy & Physiology Terminology", terminology_table_to_be_translate)

Here’s the updated table with translations, phonetic symbols, and embedded audio links for each term in the anatomical terminology:

| **Term**                | **Phonetic** | **Audio Link** | **Chinese Translation** | **Category**         | **Description**                                                                                                                                                            |
|-------------------------|--------------|----------------|-------------------------|----------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Superior (Cranial)      | /səˈpɪr.i.ər/ | [Audio](https://example.com/audio/superior.mp3) | 上方 (头侧) | Anatomical Term      | Towards the head or upper part of the body; e.g., the hand is part of the superior extremity.                                                                             |
| Inferior (Caudal)       | /ɪnˈfɪr.i.ər/ | [Audio](https://example.com/audio/inferior.mp3) | 下方 (尾侧) | Anatomical Term      | Away from the head; lower part of the body; e.g., the foot is part of the inferior extremity.                                                                              |
| Anterior (Ventral)      | /ænˈtɪr.i.ər/ | [Audio](https://example.com/audio/anterior.mp3) | 前方 (腹侧) | Anatomical Term      | Front of the body; e.g., the kneecap is located on the anterior side of the leg.                                                                                         |
| Posterior (Dorsal)      | /pɒsˈtɪr.i.ər/ | [Audio](https://example.com/audio/posterior.mp3) | 后方 (背侧) | Anatomical Term      | Back of the body; e.g., the shoulder blades are located on the posterior side of the body.                                                                                 |
| Medial                  | /ˈmiː.di.əl/ | [Audio](https://example.com/audio/medial.mp3) | 媒体的 | Anatomical Term      | Towards the midline of the body; e.g., the middle toe is located at the medial side of the foot.                                                                          |
| Lateral                 | /ˈlæ.tə.rəl/ | [Audio](https://example.com/audio/lateral.mp3) | 外侧 | Anatomical Term      | Away from the midline of the body; e.g., the little toe is located at the lateral side of the foot.                                                                       |
| Proximal                | /ˈprɒks.ɪ.məl/ | [Audio](https://example.com/audio/proximal.mp3) | 近端 | Anatomical Term      | Nearest to the trunk or point of origin; e.g., the proximal end of the femur joins with the pelvic bone.                                                                    |
| Distal                  | /ˈdɪs.təl/ | [Audio](https://example.com/audio/distal.mp3) | 远端 | Anatomical Term      | Farthest from the trunk or point of origin; e.g., the hand is located at the distal end of the forearm.                                                                   |
| Coronal Plane           | /kəˈroʊ.nəl/ | [Audio](https://example.com/audio/coronal_plane.mp3) | 冠状面 | Anatomical Term      | A vertical plane that divides the body into anterior and posterior portions.                                                                                               |
| Sagittal Plane          | /ˈsædʒ.ɪ.təl/ | [Audio](https://example.com/audio/sagittal_plane.mp3) | 矢状面 | Anatomical Term      | A vertical plane that divides the body into right and left sides.                                                                                                        |
| Axial Plane (Transverse)| /ˈæk.si.əl/ | [Audio](https://example.com/audio/axial_plane.mp3) | 轴向面 (横面) | Anatomical Term      | A horizontal plane that divides the body into upper and lower parts.                                                                                                      |
| Median Plane            | /ˈmiː.di.ən/ | [Audio](https://example.com/audio/median_plane.mp3) | 中线 | Anatomical Term      | A sagittal plane that divides the body into equal right and left halves.                                                                                                  |
| Thoracic Cavity         | /θɔːˈræs.ɪk/ | [Audio](https://example.com/audio/thoracic_cavity.mp3) | 胸腔 | Anatomical Term      | Upper ventral cavity that contains the heart, lungs, trachea, esophagus, and large blood vessels.                                                                          |
| Abdominal Cavity        | /əbˈdɒm.ɪ.nəl/ | [Audio](https://example.com/audio/abdominal_cavity.mp3) | 腹腔 | Anatomical Term      | Contains most of the gastrointestinal tract, kidneys, and adrenal glands; bound cranially by the diaphragm.                                                                |
| Pelvic Cavity           | /ˈpɛl.vɪk/ | [Audio](https://example.com/audio/pelvic_cavity.mp3) | 盆腔 | Anatomical Term      | Contains the urogenital system and the rectum; bound cranially by the abdominal cavity and dorsally by the sacrum.                                                        |
| Dorsal Cavity           | /ˈdɔːr.səl/ | [Audio](https://example.com/audio/dorsal_cavity.mp3) | 背腔 | Anatomical Term      | Smaller cavity containing organs lying posteriorly; divided into cranial cavity (containing the brain) and vertebral canal (containing the spinal cord).                 |
| Heart                   | /hɑːrt/ | [Audio](https://example.com/audio/heart.mp3) | 心脏 | Anatomical Term      | The muscular organ that pumps blood throughout the body.                                                                                                                  |
| Lungs                   | /lʌŋz/ | [Audio](https://example.com/audio/lungs.mp3) | 肺 | Anatomical Term      | Organs involved in breathing; facilitate gas exchange.                                                                                                                    |
| Kidneys                 | /ˈkɪd.niz/ | [Audio](https://example.com/audio/kidneys.mp3) | 肾脏 | Anatomical Term      | Organs that filter blood to produce urine; involved in maintaining fluid and electrolyte balance.                                                                          |
| Testes                  | /ˈtɛs.tiz/ | [Audio](https://example.com/audio/testes.mp3) | 睾丸 | Anatomical Term      | Male reproductive organs that produce sperm and hormones.                                                                                                                 |
| Ovaries                 | /ˈoʊ.və.riz/ | [Audio](https://example.com/audio/ovaries.mp3) | 卵巢 | Anatomical Term      | Female reproductive organs that produce eggs and hormones.                                                                                                                |
| Endocrine Glands        | /ˈɛn.də.kraɪn/ | [Audio](https://example.com/audio/endocrine_glands.mp3) | 内分泌腺 | Anatomical Term      | Glands that secrete hormones directly into the bloodstream; includes pituitary, thyroid, adrenal glands, etc.                                                              |
| Neurons                 | /ˈnjʊə.rɒnz/ | [Audio](https://example.com/audio/neurons.mp3) | 神经元 | Anatomical Term      | Nerve cells that transmit nerve impulses throughout the nervous system.                                                                                                    |
| Epithelial Tissue       | /ˌɛp.ɪˈθɪl.i.əl/ | [Audio](https://example.com/audio/epithelial_tissue.mp3) | 上皮组织 | Tissue Type          | A type of tissue that forms protective layers on body surfaces and linings of cavities.                                                                                  |
| Connective Tissue       | /kəˈnɛk.tɪv/ | [Audio](https://example.com/audio/connective_tissue.mp3) | 结缔组织 | Tissue Type          | Tissue that supports, binds together, and protects tissues and organs of the body; includes bone, adipose, and blood.                                                     |
| Muscle Tissue           | /ˈmʌs.əl/ | [Audio](https://example.com/audio/muscle_tissue.mp3) | 肌肉组织 | Tissue Type          | Tissue responsible for movement; includes skeletal, cardiac, and smooth muscle.                                                                                           |
| Nervous Tissue          | /ˈnɜːr.vəs/ | [Audio](https://example.com/audio/nervous_tissue.mp3) | 神经组织 | Tissue Type          | Tissue that makes up the brain, spinal cord, and nerves; responsible for transmitting signals throughout the body.                                                         |

This table incorporates the translations and additional columns as requested. Ensure to replace the audio links with the actual links to the audio files for proper functionality.